#### INIT

In [5]:
import os
import pickle
import time
from collections import Counter

import cv2
import face_recognition
from imutils import paths
from IPython.display import clear_output

TRAINING_DATASET = 'dataset_training'
TESTING_DATASET = 'dataset_testing'
DATASET_FILE_NAME = 'model_encodings.pickle'

FONT = cv2.FONT_HERSHEY_SIMPLEX

### Database connector

In [ ]:
DB_CONNECT = mysql.connector.connect(
    host="localhost",
    port="3308",
    user="root",
    passwd=""
)
IMG_SIZE = 128

mycursor = DB_CONNECT.cursor()
mycursor.execute("USE `student_record`")

### Encode

In [6]:
print("[INFO] fetching images...")
image_paths = list(paths.list_images(TRAINING_DATASET))

knownEncodings = []
knownNames = []

num_images = len(image_paths)

for (i, image_path) in enumerate(image_paths):
    name = image_path.split(os.path.sep)[-2]
    print("[INFO] processing image (%s) %d/%d..." % (name, i + 1, num_images), end="\r")

    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    boxes = face_recognition.face_locations(image, model='cnn')

    encodings = face_recognition.face_encodings(image, boxes)

    for encoding in encodings:
        knownEncodings.append(encoding)
        knownNames.append(name)
 
print("\n[INFO] serializing encodings...")
data = {"encodings": knownEncodings, "names": knownNames}

f = open(DATASET_FILE_NAME, "wb")
f.write(pickle.dumps(data))
f.close()

print("[INFO] DONE.")

[INFO] fetching images...
[INFO] processing image (vaibhav_pathak) 228/228...
[INFO] serializing encodings...
[INFO] DONE.


### Live Demo

In [ ]:
print("[INFO] loading encodings...")
data = pickle.loads(open(DATASET_FILE_NAME, "rb").read())

print("[INFO] recognizing faces...")

cam = cv2.VideoCapture(0)
cam.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

cv2.namedWindow("recognize faces")

num_names = Counter(data['names'])
present_name = list()

while True:
    ret, image = cam.read()
    if not ret:
        break
    k = cv2.waitKey(1)

    if k % 256 == 27:
        print("\nEscape hit, closing...")
        break

    clear_output(wait=True)

    boxes = face_recognition.face_locations(image, model='hop')
    encodings = face_recognition.face_encodings(image, boxes)
    names, present_name = [], []

    # find faces    
    for encoding in encodings:
        matches = face_recognition.compare_faces(data["encodings"], encoding)
        num_prop = 0

        if True in matches:
            matchedIdxs = [i for (i, b) in enumerate(matches) if b]
            counts = {}

            for i in matchedIdxs:
                name = data["names"][i]
                counts[name] = counts.get(name, 0) + 1

            name = max(counts, key=counts.get)
            num_prop = counts[name] / num_names[name]

            print("[INFO] detected %s with '%f' accuracy ..." %
                  (name, num_prop))

        if(num_prop > 0.85):
            t = time.strftime('%Y-%m-%d %H:%M:%S')

            if name not in present_name:
                q = "INSERT INTO attendance(name, timestamp) values ('" + \
                    name + "', '" + t + "');"
                mycursor.execute(q)

            print("[INFO] Found %s (%f)." % (name, num_prop))

            present_name.append(name)

        names.append(name if num_prop > 0.85 else "Unknown")

    # draw rectangle over found faces
    for ((top, right, bottom, left), name) in zip(boxes, names):
      
        left, top, right, bottom = left - 40, top - 80, right + 40, bottom + 20
        
        cv2.rectangle(image, (left, top), (right, bottom), (255, 0, 0), 2)
        
        s = cv2.getTextSize(name, FONT, 0.75, 1)

        cv2.rectangle(image, (left, top), (left + s[0][0] + 40,  top + s[0][1] + 20), (255, 0, 0), -1)
        name = name.replace("_", " ")
        name = name.title()

        cv2.putText(image, name, (left + 20, top + 10 + s[0][1]), FONT, 0.75, (255, 255, 255), 2)


    cv2.imshow("recognize faces", image)

cam.release()

cv2.destroyAllWindows()